In [2]:
%load_ext tutorial.tests.testsuite

The tutorial.tests.testsuite extension is already loaded. To reload it, use:
  %reload_ext tutorial.tests.testsuite


# Functional Programming in Python

## Presentation slides
The following section contains the slides shown during the workshop

## Functional programming in python
Why functional programming?
- easy **reasoning*: *there are no surprises*
- suitable for **parallel** programming
- breaking down a large problem in smaller problems


## The principles of functional programming

- *Functions as values*
- *Pure functions*
- *Immutability of data*
- *Composition*
- *Referential  transparency*
- *Type system*

## Functions as values

Functions are *objects* of the programming language.  They can be assigned and manipulated like other values:



In [2]:

def x()-> int:
    return 3

y = x




This defines a zero-argument function `x` that returns an integer.
We then assign the value of `x` to `y` and check whether they are the same function

In [3]:
print(x() == y())

True


Success! We see that functions are values and can be used as any other value

## Higher order functions

The fact that functions are values leads to the use *higher-order-functions*, which take other functions as parameter. This idiom is useful to abstract over the behavior of programs.




For example, `map` is used to apply a function to a list of values:

In [4]:
from IPython.display import Markdown
import inspect
def my_fun(a: int)-> int:
    return a + 1

input_list = list(range(10))
result = list(map(my_fun, input_list))



In [5]:
print(f"Input: {input_list}, output: {result}")

Input: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


We can create *anonymous* functions using `lambda`. These are functions that do not have a name like in `def`. We can name them by assigning them to a variable:

In [6]:
new_fun = lambda x: x + 1

In [7]:
new_fun(5)

6

## Pure functions

We prefer functions that have no *side-effects*: 

1. they do not modify the state of the program 
2. given the same input, they **always** return the same output.



This following function is not *pure* because it modifies the value of `x` every time it runs:


In [8]:
from typing import Any
x = [1,2,3]
def do_something(val: int) -> list[int]:
    return x.append(val)

def display_val(x: Any):
    print(f"{x=}, {id(x)=}")


In [9]:
display_val(x)
do_something(1)
display_val(x)
do_something(2)
display_val(x)

x=[1, 2, 3], id(x)=140510397969920
x=[1, 2, 3, 1], id(x)=140510397969920
x=[1, 2, 3, 1, 2], id(x)=140510397969920


`id(x)` returns the memory address of a variable. If it does not change, it means the variable refers to the same object. This is the case here, therefore we proved that `do_something` is not *pure*.

## Immutable Data

Whenever possible, we avoid *mutating* data, as mutation is a type of *side-effect* and leads to unexpected behavior. 




- When a variable is defined, its value **remains constant for the duration of the program execution**.

- This makes it easier to reason about the flow of our program. 

For example, instead of inserting an element in an existing list as we did before, we create a new list by combining the old list with the new element:

In [10]:
x = [1, 2, 3]
print(f"{x=}")
y = x + [4,]
print("Appending")
print(f"{x=}")
print(f"{y=}")

x=[1, 2, 3]
Appending
x=[1, 2, 3]
y=[1, 2, 3, 4]


As you can see, the value of `x` does not change when we create `y`. This is in contrast to the previous example.

## List comprehensions
Immutability of data in python is frequently expressed using *comprehensions*.
These are for-like expressions that operate on iterables and produce *values* instead of modifying existing variables (no side effects).



To see how this works, let's first look at an example of *mutation*: we want to produce a list of the first 10 squares:

In [11]:
#We define an empty list
a = []
#range(n) creates an iterable from 0,...,n
for i in range(10):
    #We append the current square to the existing list
    a.append(i**2)
    print(f"{a=}, {id(a)=}")

a=[0], id(a)=140510397970816
a=[0, 1], id(a)=140510397970816
a=[0, 1, 4], id(a)=140510397970816
a=[0, 1, 4, 9], id(a)=140510397970816
a=[0, 1, 4, 9, 16], id(a)=140510397970816
a=[0, 1, 4, 9, 16, 25], id(a)=140510397970816
a=[0, 1, 4, 9, 16, 25, 36], id(a)=140510397970816
a=[0, 1, 4, 9, 16, 25, 36, 49], id(a)=140510397970816
a=[0, 1, 4, 9, 16, 25, 36, 49, 64], id(a)=140510397970816
a=[0, 1, 4, 9, 16, 25, 36, 49, 64, 81], id(a)=140510397970816


`id(a)` returns the memory *location* of `a`. As you can see, we modify the contents of `a` over the for loop: a always refers to the same location in the memory and therefore to the same object.

Let's rewrite this using a compherension and see what happens:

In [12]:
#range(n) creates an iterable from 0,...,n
a = [i for i in range(10)]
print(f"{a=}, {id(a)=}")
b = [i**2 for i in a]
print(f"{b=}, {id(b)=}")
print(f"{a=}, {id(a)=}")

a=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], id(a)=140510395940288
b=[0, 1, 4, 9, 16, 25, 36, 49, 64, 81], id(b)=140510395940480
a=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], id(a)=140510395940288


We see that the second expression produced a *new list* that we assigned  to `b`. The value of `a` remained unchanged.

## Composition
We build our programs by *composing* many simple functions togehter.

Why:


- It is easier to fix simple functions
- It is easier to optimise the program
- Single responbiility: every function only does one job
- ...

How do we do this?

In [13]:
# This is not ideal
def stupid_long_function(x: int) -> int:
    x1 = x + 2
    x2 = x1 - 4
    x3 = x2**2
    return x3

Instead of running all computation in a single long function, we can do better by *composing* many small functions.
 

In [14]:
def f1(x: int) -> int:
    return x + 2

def f2(x: int) -> int:
    return x - 4

def f3(x: int) -> int:
    return x ** 2

# `better_function` is the "composition" of f3, f2, f1
better_function = lambda x: f3(f2(f1(x)))



In [15]:
better_function(3) == stupid_long_function(3)

True

Instead of writing a long function, we broke it down into smaller functions and reconstructed it by composing them together. 

## Referential transparency
This sounds scary, but it simply means that whenever we see a function call (or any expression), we can subsitute it with the value of the function.

This is the essence of functional programming: *there are no surprises*. We can reason about the flow of our program 

Consider this example:

In [16]:

def fun(x: int) -> int:
    return x + 2

y1 = fun(1) + 3


x_out = fun(1)
y2 = x_out + 3

print(f"{y1=}")
print(f"{y2=}")


y1=6
y2=6


We see that `x` is indeed referentially transparent. Now, look at another example:

In [17]:
import random
def x2() -> int:
    #This generates a random integer between 0 and 10
    return random.randint(0, 10)

# This "captures" the expression x() + 3 without computing its value by defining an "anonymous function"
# We get the value by calling y1()
y1 = lambda : x2() + 3





In [18]:
print(f"{y1()=}")
print(f"{y1()=}")

y1()=13
y1()=4


This is *not*. Every time we call `x2`, we get a different (random) value. 



In general, any expression containing a *combination of pure functions* and no side-effect is referentially transparent. We try to write our programs by composing referentially transparent functions as much as possible.

Why:
- We can reason about the program flow
- Compilers and other tools can reason for us and help finding errors
- It is easier to optimise our programs

## Type system

A typical trait of (modern) functional programming, is the heavy use of the *type system*.
This provides a set of rules to classify expression and values in the language into classes called *types*.
You encountered some basic types in the introduction, for example `int`, `str` or `float`.


We use type systems to enforce constraints and invariants in our program:

- *Making illegal states unrepresentable*
- Making sure we pass the right input to functions
- Checking if we wrote a valid program before it runs



Modern (3.6+) python allows using *type annotation* to mark the type of values, function parameters and class members.

For example:

```
def fun(a: int) -> str:
    return f"{a}"
```

means that `fun` takes integer values and returns strings. Python *does not enforce* this constraint. There are tools however, which can be used to check the conistency of types in a python program.

For example, instead of doing:

In [19]:

def greet(name: str, age: int):
    print(f"Hello {name}, you are {age} years old")
    


In [20]:
greet("Simone", 34)

Hello Simone, you are 34 years old


Because python lacks static type checking, we can do absurd things:

In [21]:
greet("Simone", "age")

Hello Simone, you are age years old


We can do better:

In [22]:

#This defines a simple class with some helper methods and validations
class Person:
    name: str
    age: int
    def __init__(self, name: str, age: int):
        if not isinstance(name, str) or not isinstance(age, int):
            raise ValueError("Invalid input")
        self.age = age
        self.name = name

def greet_better(person: Person):
    print(f"Hello {person.name}, you are {person.age} years old")


In [23]:
greet("Simone", 34)
try:
    greet_better(Person("Simone", "d"))
except:
    print("Ooops, it does not work")

Hello Simone, you are 34 years old
Ooops, it does not work


## Writeup

## References
Here are some additional references to help you in self-learning. Next to each link, we write if this is a video, text or another type of material

- The [functools reference](https://docs.python.org/3/library/functools.html) from the python standard library (text)
- The [itertools reference](https://docs.python.org/3/library/itertools.html) from the python standard library (text)
- [Functional programming howto](https://docs.python.org/3/library/functools.html) from the python documentation
- A [good, but old introduction](https://www.youtube.com/watch?v=Ta1bAMOMFOI) of functional programming (video)
- A [very high level](https://www.youtube.com/watch?v=Qa8IfEeBJqk) introduction of functional programming (video, advanced). Interesting, but not python-specific as it refers to Haskell
- A [general introduction on functional programming](https://www.youtube.com/watch?v=8z_bUIl_uPo). Very worth watching as it uses python for the examples (video)
- [Principles of functional programming](https://dev.to/jamesrweb/principles-of-functional-programming-4b7c)



## Introduction
Functional programming is an approach to programming where programs are built by composing and running functions that perform a series of transformation on data. This is in contrast with the common approach of *imperative programming*, where programs are written as series of statements which modify the *state* of the computation environment. Normally within functional programming, great focus is placed on *composability*, *immutability* and  *purity*. We are going to define these terms in more detail later.
## Why Functional Programming
Why do we choose functional programming? There are a series of advantages to this approach, namely:
- **Debugging** and **testing** are easy: there are no surprises because every function only does one thing and does not affect any other piece of the program.
- **Parallelisation** is trivial: because functions are just small boxes that take one input and produce an output and do not depend on other parts of the code in an implicit manner through global variables or other shared pieces of state, it is easy to make several functions run in parallel.
## The basic principles of functional programming
All modern programming languages have *functions* (or methods, procedures, subroutines, subprograms); these are groups of program statements that perform a certain computation. Functions are defined once for the whole program and can be reused at will throughout the program whenever we need to perform the specific computation they are defined for. Using functions, we can split our code in smaller units that are only responsible for a specific *functionality*; this helps us structuring our code in a clean and understandable form. 

The main principles we use in functional programming are:
- *purity*
- *functions as values* (*higher order functions*)
- *immutability of data*
- *composition*
- *referential transparency*

In the next few sections, we are going to briefly explore these concepts with some examples.


### Pure Functions (Purity)
In functional programming, we try to strive for *purity*, that is we want to define and use functions that only depend on their input, always return the same output for the same inputs and do not have any *side effects*, that is they do not indirectly affect any other part of our program. You can think of these functions as mathematical functions. 
Other examples of side effects are:
- printing to the program output
- reading or writing files
- generating and using random numbers
To better understand this concept, let us look at the function `my_first_pure_function` we defined below:


In [24]:
def my_first_pure_function(x: int) -> int: 
    return x + 1

&#x1F600
Is this a pure function?
As you could probably imagine, this function is pure. Any time we run it, we get the same  result and running the function does not affect any other part of our program:

In [25]:
x = 1
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")

x is 1
The result of calling the function is 2
x is 1
The result of calling the function is 2
x is 1


As a rule of thumb, any function that does not modify variables outside of its *scope* and only uses mathematical functions is a pure function.
Now, consider this function instead:

In [26]:
x = ["short", "list"] 
def do_something(y: str) -> None:
    x.append(y)

Is this function pure? 

Let's try and run it. 

In [27]:
print(x)
do_something("a")
print(x)

['short', 'list']
['short', 'list', 'a']


As you see from the output, the function modified the list `x`. Therefore, this function is not pure. This leads us to the next principle, **immutability**.

### Immutability
When writing programs in functional style, we usually avoid functions like `do_something`. Instead of modifying existing data (*mutation*), you write functions that transform your data and return new objects.
In the case of the function above, we would rewrite it as follows:


In [28]:
x = ["short", "list"] 
def do_something_immutable(x: "list[str]", y: str) -> "list[str]":
    return x + [y]

print(x)
print(do_something_immutable(x, "a"))
print(x)

['short', 'list']
['short', 'list', 'a']
['short', 'list']


The output shows that `do_something_immutable` does not change `x`. It also does not reference to `x` outside of the scope but takes it as an argument. Instead of modfying the original `x` list, it returns a new list. When we want to keep immutability, this is the style we work with. If we adopt this style, it is easier to reason about the flow of our program as there are no variable that are being modified *at distance* by other function calls.

### Composition
Another important aspect of functional programming is `composition`. This means that whenever we want to perform multiple operations, we avoid writing intermediate variables, especially so when these are in the global state of the program.

Instead, we design our functions in such a way that one functions output can be directly fed into the next function.

Additionally, --but this is not as useful in python-- we use the associative property of function application:

```
f(g(h)) = f . g (h)
```
where we use the `.` to express the composition of functions.

In python, we can do this using a `lambda` expression:



In [29]:
def f(x: int) -> int:
    return x + 4

def g(x: int) -> int:
    return x * 3

fg = lambda x: f(g(x))

print(f"{fg(4)=}, {f(g(4))=}")

fg(4)=16, f(g(4))=16


In this way, we can break down a complex calculation in a series of simpler calculations. This is useful for many things:

- It is easier to find problems in a smaller function
- We do our work only once: every time we need the same action, we just call the same function

### Higher Order Functions / Functions as Values
Another important principle of functional programming is that **functions are values**. In programming languages (like python) that support a functional style, we can manipulate functions with the language, we can pass them around in a variable and even use functions as parameters for another function.

As an example, consider the function `function_caller`:



In [30]:
from typing import Callable, Any
def function_caller(f: Callable[[Any], Any], arg: Any) -> Any:
    print(f"Calling the function {f} with argument {arg}")
    res = f(arg)
    print(f"The result of the call was {res}")
    return res

This function takes a function `f` and its argument `args` as input and return the result of calling the function, while additionally printing a message on the standard output of the program. Let's try this out.
To do so, we define a new function `add_five`:

In [31]:
def add_five(x: int) -> int:
    return x + 5

Now we can call `function_caller` with `add_five` and another number as arguments:

In [32]:
function_caller(add_five, 4)

Calling the function <function add_five at 0x7fcb203d80e0> with argument 4
The result of the call was 9


9

This example was a bit convoluted, but it shows that in python we can use functions as values and even pass them as arguments to other functions.

This is actually useful in many cases; a typical example being numerical optimization, where we want to find the parameters to a function that minimize a certain criteria.

Other than these specific use cases, there are some common *higher order functions*  (HoF), or functions that take functions as parameters that are common in most programming languages.  We will look at a few in the next section.



### Referential transparency
The last principle we like to discuss is (*referential transparency*)[https://en.wikipedia.org/wiki/Referential_transparency]. This complex-sounding name simply means that we can replace any expression with its value without changing the behavior of the program.
Consider this program:

In [33]:
def x() -> int:
    return 5

y1 = x() + 5
y2 = x() + 5

x() + 5 == y1

y1 == y2


True

If `x` is referentially transparent, we can replace its value in the expression `y1 = x() + 5` and we obtain the same value. In other words, the value of `y` does not change if we execute it again, as shown in the code above: we have two expressions `y1` and `y2` that use the same code and because `x` is referentially transparent, 
their value is equal.

In this case, this is true because the value of `x()` is simply 5 and is invariant.

However, consider this counterexample:


In [34]:
#This is used to generate random number
import random
def x() -> int:
    return random.randint(0, 10)


# With lamba we capture the expression x() + 5 in a function,
# we get its value when we call it like a function
y1 = lambda: x() + 5

y2 = lambda: x() + 5

y1() == y2()

False

in this case, because the function returns a different random number every time we call it, the expression `y1` has a different *value* compared to `y2` even though they both have the same *form*.



## Mapping / Iteration 

In this section, we will look at the application of some of the principles stated above in python. An important application of functional programming in python is the manipulation of iterables and lists. 

### Using map and filter

Python offers the `map` function which can be used to apply a given function to all members of an `iterable`. Do not worry too much about what an `iterable` exactly is, just see it as a generalisation of a list, representing all objects that we can `iterate` over, accessing their values one by one. Let's see an example of `map` using the `add_five` function:

In [35]:
map(add_five, [1,2,3,4])

The output is a bit confusing! The reason is that `map` does not return a list but a [map](https://docs.python.org/3/library/functions.html#map) object. This object is `lazy`, that means that the values are only generated when we access them, for example by iterating over the `map` object. 
Fortunately, we can easily convert this into a list by calling the `list` function:

In [36]:
list(map(add_five, [1,2,3,4]))

[6, 7, 8, 9]

This is equivalent to using a for loop:


In [37]:
a = []
for i in [1,2,3,4]:
    a.append(add_five(i))
print(a)

[6, 7, 8, 9]


However, as you can see this former style is not very functional because we mutate an existing list `a` by appending the results of calling `add_five` to it. We will see later that python offers a syntax called *list comprehension* to express this operation in a more functional style.
`map` is very useful when we want to apply the same functions to a list of parameters and we do not want to write a `for` loop. It makes for more coincise code, while sacrificing some readability. 

#### Filtering
Another basic higher order function is `filter`. As the name says, this function is used to filter an `iterable` using a *predicate function*. This is a function that takes a value and return `true` or `false`. With `true`, the current element is kept, with `false` it is discarded.
We now try to write a predicate function that only keeps even numbers.

<a name="filter-example"></a>

In [38]:
def is_even(x: int) -> bool:
    return x % 2 == 0

Now we are ready to try `filter` using `is_even`. Because `filter` returns a `filter` object, we wrap it in `list` to directly see the result as a list:

In [39]:
list(filter(is_even, [1,2,3,4,5]))

[2, 4]

#### Reducing
A third basic HoF is *reduction*. This is a function that takes a function `f(x, y)` of two arguments and an iterable `it` and applies the function  to every element in the iterable *cumulatively* to produce one value. It works in the following way:
At the first step:

- The first argument of `f`, `x` is the current value of the accumulation. At first, this corresponds to the first element of `it`
- The second argument of `f`. `y` is the current element of the iterable. At first, this corresponds to the second element of `it`.

Because of this behavior, this function is useful to compute sums or similar aggregations over a list. In python, this function is available in the `functools` [module](https://docs.python.org/3/library/functools.html) as a part of the python standard library.

As an example of using `reduce`, consider the following snippet:


In [63]:
import functools


def spy(x: int, y: int) -> int:
    val = x * y
    print(f"x: {x}, y:{y}, result: {val}")
    return val

result = functools.reduce(spy, [1, 2, 3, 4, 5])
print(result)

x: 1, y:2, result: 2
x: 2, y:3, result: 6
x: 6, y:4, result: 24
x: 24, y:5, result: 120
120


As you can see, we are computing the cumulative product of `[1, 2, 3, 4, 5]`, that is the *factorial* of 5, or `5!`.
The print in `spy` helps us seeing what goes on inside of the function at each step in reduce.

### List comprehensions
Many of the operations in the previous section can be performed in a different (some would say more *pythonic*) way using [list comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).
These look like mini for-loops inside square brackets and are very useful to write programs in a functional style while keeping code more readable than using higher order functions.  


For example, if we want to double all integers between 1 and 10 and store the result in a list,  we can proceed in the classical imperative way:

In [41]:
#create an empty list
doubles = []
#every time the loop runs, we add a number to the list
for i in range(10):
    doubles.append(i * 2)

print(f"The doubles are {doubles}")


The doubles are [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Or we can use `map` like this:

In [42]:
doubles = map(lambda x: x * 2, range(10))

print(list(doubles))


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Again here we wrapped `doubles` in `list` because `map` is lazy and does not return a list immediately but a `map` object which can be converted into a list using `list`. 
This solution is more elegant and avoids side effects because we do not modify the previously defined `doubles` list, instead we directly produce a new list with the desired result.

Now, we solve the same problem using a list comprehensions:

In [43]:
doubles = [i * 2 for i in range(10)]
print(doubles)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


This looks a lot like the solution above for the for loop, but it is an *expression* instead of a *statement*. [Expressions](https://www.baeldung.com/cs/expression-vs-statement) in programming are pieces of code that return a value and hence can be composed together to form more complex expressions and used anywhere a value is needed. In functional style, we prefer working with expressions instead of statements because we try to keep [*referential transparency*](https://en.wikipedia.org/wiki/Referential_transparency).

Compherensions can do more than just iterate over a list. We can use them to filter elements of a list by adding an `if` statement at the end.
For example, consider the example of filtering even numbers  from a list [above](#filter-example); we can rewrite this with list compherensions like this:


In [44]:
def is_even(x: int) -> bool:
    return x % 2 == 0

[i for i in range(5) if is_even(i)]

[0, 2, 4]

We can also combine (*nest*) list compherensions. This is equivalent to a nested for loop. For example consider this expression:

In [45]:
[(i,j) for i in range(5) for j in range(5)]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

This returns all the combination of numbers between 0 and 4 as a list of pairs. In the first part of the compherension, we can use any expression instead of just referencing the variable. For example we can return a description of the pair `i,j` in text:

In [46]:
[f"The current pair is {(i,j)}" for i in range(5) for j in range(5)]

['The current pair is (0, 0)',
 'The current pair is (0, 1)',
 'The current pair is (0, 2)',
 'The current pair is (0, 3)',
 'The current pair is (0, 4)',
 'The current pair is (1, 0)',
 'The current pair is (1, 1)',
 'The current pair is (1, 2)',
 'The current pair is (1, 3)',
 'The current pair is (1, 4)',
 'The current pair is (2, 0)',
 'The current pair is (2, 1)',
 'The current pair is (2, 2)',
 'The current pair is (2, 3)',
 'The current pair is (2, 4)',
 'The current pair is (3, 0)',
 'The current pair is (3, 1)',
 'The current pair is (3, 2)',
 'The current pair is (3, 3)',
 'The current pair is (3, 4)',
 'The current pair is (4, 0)',
 'The current pair is (4, 1)',
 'The current pair is (4, 2)',
 'The current pair is (4, 3)',
 'The current pair is (4, 4)']

### Combinining and splitting iterators
Sometimes, we need to iterate over multiple lists in a special fashion, for example over two lists in parallel or we want to compute all the combinations of elements of two lists etc.
We can solve many ot these problems with the use of list compherensions, but sometimes there are more elegant solutions by using the [`itertools`](https://docs.python.org/3/library/itertools.html) module, a part of the python standard library.

Lets see some example of problems we can solve by using these tools.

- Iterating over two lists in parallel: 
    consider the two lists `numbers = [1, 2, 3, 4, 5]` and `words = ["one", "two", "three" , "four", "five"]`. We want to produce a list of strings consisting of the content of `numbers` and `words`, such that each number in `numbers` corresponds to the right word in `words`. We can do this by using the `zip` buil-in function:
`zip` takes a number of `iterables` as an input and returns a list of tuples. The i-th element of this list is a tuple with the i-th element of each iterable, proceeding until the shortes iterable is exhausted. Using `zip`, we solve our problem like this:

In [47]:
numbers = [1, 2, 3, 4, 5]
words = ["one", "two", "three" , "four", "five"]
[f"The number {i} is called {j}" for i, j in zip(numbers, words)]

['The number 1 is called one',
 'The number 2 is called two',
 'The number 3 is called three',
 'The number 4 is called four',
 'The number 5 is called five']

-  Producing permutations of elements.
    Suppose we have a list containing the letters `["A", "B", "C", "D", "E"]` and we want to produce all four letter words from them. We can use   `itertools.permutations`. Once again, we wrap the operation in `list` to obtain a list as output:

In [67]:
import itertools
letters = ["A", "B", "C"]
list(itertools.permutations(letters, 3))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

Another useful trick to be aware of when working with lists is [`unpacking`](https://docs.python.org/3/tutorial/controlflow.html#tut-unpacking-arguments). In python, we can extract elements from a list using the assignment statement.  For example, if we have a two-element list, we can write

In [49]:
a, b = [1, 2]

and extract the two elements in the variables `a` and `b`. If you try the same statement with a three-element list you will get an error:

In [50]:
try:
    a, b = [1, 2, 3]
except ValueError:
    print("It does not work")

It does not work


However, you can extract *all remaining elements* with the star notation:

In [51]:
a, *b = [1, 2, 3]

If we use `*` before an argument in a function call, this will unpack the argument and pass each component to the function separately, as you can see in this example:

In [52]:
def print_variables(a: any, b: any, c: any):
    print(f"a is {a}, b is {b}, c is {c} ")

print_variables(*[1, 2, 3])
print_variables(1, 2, 3)
print_variables(*[[1, 2, 3], [4, 5, 6], [5, 6, 7]])

a is 1, b is 2, c is 3 
a is 1, b is 2, c is 3 
a is [1, 2, 3], b is [4, 5, 6], c is [5, 6, 7] 


Be careful, this only works if the list/iterable you unpack is as long as the arguments you want to pass to your function.



## Exercises


### Exercise 1: Pure or impure functions? 🌶️

For each of the functions below, determine whether they are pure or impure

In [53]:
def fun1(a: "list[str]") -> None:
    a.append("b")


def fun2(a: int) -> int:
    return a + 2


def fun3(a: "dict[str, str]"):
    a["test"] = "dest"



### Exercise 2: Keeping only multiples of n 🌶️

Given a list `L` of integers, write a function that only keeps the numbers that are multiples of a given constant `k`.

- Example 1: given `nums = [1, 2, 3, 4, 5]`, and `k = 2`, the result must be `[2, 4]`

- Example 2: given `nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] ` and `k = 5`, the result is `[5, 10]`

Write your function in the cell below inside of the `solution_exercise1` function. The function receives a list `l` as an input and should return another list

In [54]:
%%ipytest functional_programming
def solution_exercise2(l: "list[int]", k: int):
    """
    Write your solution here
    """
    pass

FF                                                                       [100%]
=================================== FAILURES ===================================
_____ test_exercise2[solution_exercise2-l0-1-reference_solution_exercise2] _____
tutorial/tests/test_functional_programming.py:27: in test_exercise2
    assert function_to_test(l, k) == reference_func(l, k)
E   assert None == [1, 2, 3, 4]
E    +  where None = <function solution_exercise2 at 0x7fcb203ff380>([1, 2, 3, 4], 1)
E    +  and   [1, 2, 3, 4] = <function reference_solution_exercise2 at 0x7fcb201da200>([1, 2, 3, 4], 1)
_____ test_exercise2[solution_exercise2-l1-5-reference_solution_exercise2] _____
tutorial/tests/test_functional_programming.py:27: in test_exercise2
    assert function_to_test(l, k) == reference_func(l, k)
E   assert None == [0, 5, 10, 15, 20, 25, ...]
E    +  where None = <function solution_exercise2 at 0x7fcb203ff380>([0, 1, 2, 3, 4, 5, ...], 5)
E    +  and   [0, 5, 10, 15, 20, 25, ...] = <function refer

### Exercise 3: Transposing a Matrix 🌶️🌶️

Consider a matrix `M` represented row-wise as a list of lists `[[1, 2, 3], [4, 5, 6], [7, 8, 8]]`.
Write a function  that returns the transpose of `M`, the matrix obtained by exchanging rows and columns

- Example 1: given `M=[[1, 0], [0, 1]]`, the result must be `[[1, 0], [0, 1]]`
- Example 2: given `M=[[1, 2, 3], [4, 5, 6], [7, 8, 8]]` the result must be `[1, 4, 7], [2, 5, 8], [3, 6, 8]]`

In [3]:
%%ipytest functional_programming

def solution_exercise3(m: "list[list[int]]") -> "list[list[int]]":
    """
    Write your solution here
    """
    pass


FF                                                                       [100%]
=================================== FAILURES ===================================
__________________ test_exercise3[solution_exercise3-input0] ___________________
tutorial/tests/test_functional_programming.py:37: in test_exercise3
    assert function_to_test(input.tolist()) == input.transpose().tolist()
E   assert None == [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
E    +  where None = <function solution_exercise3 at 0x7f96faef9940>([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
E    +    where [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]] = <built-in method tolist of numpy.ndarray object at 0x7f96bff25170>()
E    +      where <built-in method tolist of numpy.ndarray object at 0x7f96bff25170> = array([[1., 0., 0.],\n       [0., 1., 0.],\n       [0., 0., 1.]]).tolist
E    +  and   [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]] = <built-in method tolist of numpy.ndarray object at 0x7f9

### Exercise 4: Flattening list of lists 🌶️🌶️

Imagine we receive a list of lists `l` like `[[1, 2], [3, 4]]`. Write a function that converts this list into a `flat` list like `[1, 2, 3, 4]`. 

To write your function, just complete the code in `solution_exercise4`


In [4]:
%%ipytest functional_programming

def solution_exercise4(l: "list[list[any]]") -> "list[any]":
    """
    Write your solution here
    """
    pass



FF                                                                       [100%]
=================================== FAILURES ===================================
____ test_exercise4[solution_exercise4-input0-reference_solution_exercise4] ____
tutorial/tests/test_functional_programming.py:56: in test_exercise4
    assert function_to_test(input) == reference_func(input)
E   assert None == [1, 2, 3, 4, 4, 5, ...]
E    +  where None = <function solution_exercise4 at 0x7f96bff81260>([[1, 2, 3, 4], [4, 5, 5], [4, 5, 6]])
E    +  and   [1, 2, 3, 4, 4, 5, ...] = <function reference_solution_exercise4 at 0x7f96f00eb240>([[1, 2, 3, 4], [4, 5, 5], [4, 5, 6]])
____ test_exercise4[solution_exercise4-input1-reference_solution_exercise4] ____
tutorial/tests/test_functional_programming.py:56: in test_exercise4
    assert function_to_test(input) == reference_func(input)
E   AssertionError: assert None == ['a', 'b', 'c', 'd', 'f', 'e', ...]
E    +  where None = <function solution_exercise4 at 0x7f96bff81

### Exercise 5: Counting initials 🌶️🌶️🌶️

Given a list `w` of english words, write a function that counts how many words begin with each letter of the english alphabet and returns the result as a **alphabetically sorted** list of tuples `(letter, count)`. 

Hint: consider the functions `sorted` and `itertools.groupby` from the python standard library.

You will receive the list `w` as the input of the solution skeleton below:

In [7]:
%%ipytest functional_programming
def solution_exercise5(w: list[str]) -> list[(str, int)]:
    """
    Write your solution here
    """
    print("This does not work")
    pass



F                                                                        [100%]
=================================== FAILURES ===================================
______________________ test_exercise5[solution_exercise5] ______________________
tutorial/tests/test_functional_programming.py:74: in test_exercise5
    assert function_to_test(data) == reference_solution_exercise5(data)
E   AssertionError: assert None == [('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ...]
E    +  where None = <function solution_exercise5 at 0x7f96bfe64860>(['a', 'aa', 'aaa', 'aaron', 'ab', 'abandoned', ...])
E    +  and   [('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ...] = reference_solution_exercise5(['a', 'aa', 'aaa', 'aaron', 'ab', 'abandoned', ...])
----------------------------- Captured stdout call -----------------------------
This does not work
=========================== short test summary info ============================
FAILED tutorial/tests/te

### Exercise 6: Counting initials frequency 🌶️🌶️🌶️
If you could solve the previous exercise, you now have a list `l` of the form `[('a', 20), ('b', 30), ...]`.
If you cannot, do not worry: you will receive the correct input automatically as `l` inside the function `solution_exercise6`


Write a function that computes the *relative frequency* of each letter in the list `l`. 

In [59]:
%reload_ext tutorial.tests.testsuite


In [60]:
%%ipytest functional_programming
def solution_exercise6(l: "list[(str, int)]") -> "list[(str, float)]":
    pass

FF                                                                       [100%]
=================================== FAILURES ===================================
_____ test_exercise2[solution_exercise2-l0-1-reference_solution_exercise2] _____
tutorial/tests/test_functional_programming.py:27: in test_exercise2
    assert function_to_test(l, k) == reference_func(l, k)
E   assert None == [1, 2, 3, 4]
E    +  where None = <function solution_exercise2 at 0x7fcb203ff380>([1, 2, 3, 4], 1)
E    +  and   [1, 2, 3, 4] = <function reference_solution_exercise2 at 0x7fcb201da200>([1, 2, 3, 4], 1)
_____ test_exercise2[solution_exercise2-l1-5-reference_solution_exercise2] _____
tutorial/tests/test_functional_programming.py:27: in test_exercise2
    assert function_to_test(l, k) == reference_func(l, k)
E   assert None == [0, 5, 10, 15, 20, 25, ...]
E    +  where None = <function solution_exercise2 at 0x7fcb203ff380>([0, 1, 2, 3, 4, 5, ...], 5)
E    +  and   [0, 5, 10, 15, 20, 25, ...] = <function refer

FF                                                                       [100%]
=================================== FAILURES ===================================
__________________ test_exercise3[solution_exercise3-input0] ___________________
tutorial/tests/test_functional_programming.py:37: in test_exercise3
    assert function_to_test(input.tolist()) == input.transpose().tolist()
E   assert None == [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
E    +  where None = <function solution_exercise3 at 0x7fcac75aab60>([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
E    +    where [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]] = <built-in method tolist of numpy.ndarray object at 0x7fcac7542850>()
E    +      where <built-in method tolist of numpy.ndarray object at 0x7fcac7542850> = array([[1., 0., 0.],\n       [0., 1., 0.],\n       [0., 0., 1.]]).tolist
E    +  and   [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]] = <built-in method tolist of numpy.ndarray object at 0x7fc

FF                                                                       [100%]
=================================== FAILURES ===================================
____ test_exercise4[solution_exercise4-input0-reference_solution_exercise4] ____
tutorial/tests/test_functional_programming.py:56: in test_exercise4
    assert function_to_test(input) == reference_func(input)
E   assert None == [1, 2, 3, 4, 4, 5, ...]
E    +  where None = <function solution_exercise4 at 0x7fcac7434f40>([[1, 2, 3, 4], [4, 5, 5], [4, 5, 6]])
E    +  and   [1, 2, 3, 4, 4, 5, ...] = <function reference_solution_exercise4 at 0x7fcac750cf40>([[1, 2, 3, 4], [4, 5, 5], [4, 5, 6]])
____ test_exercise4[solution_exercise4-input1-reference_solution_exercise4] ____
tutorial/tests/test_functional_programming.py:56: in test_exercise4
    assert function_to_test(input) == reference_func(input)
E   AssertionError: assert None == ['a', 'b', 'c', 'd', 'f', 'e', ...]
E    +  where None = <function solution_exercise4 at 0x7fcac7434

F                                                                        [100%]
=================================== FAILURES ===================================
______________________ test_exercise5[solution_exercise5] ______________________
tutorial/tests/test_functional_programming.py:74: in test_exercise5
    assert function_to_test(data) == reference_solution_exercise5(data)
E   AssertionError: assert None == [('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ...]
E    +  where None = <function solution_exercise5 at 0x7fcac7437ec0>(['a', 'aa', 'aaa', 'aaron', 'ab', 'abandoned', ...])
E    +  and   [('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ...] = reference_solution_exercise5(['a', 'aa', 'aaa', 'aaron', 'ab', 'abandoned', ...])
=========================== short test summary info ============================
FAILED tutorial/tests/test_functional_programming.py::test_exercise5[solution_exercise5] - AssertionError: assert None == [(

F                                                                        [100%]
=================================== FAILURES ===================================
______________________ test_exercise6[solution_exercise6] ______________________
tutorial/tests/test_functional_programming.py:84: in test_exercise6
    assert function_to_test(input_data) == reference_solution_exercise6(input_data)
E   AssertionError: assert None == [('a', 0.072), ('b', 0.0536), ('c', 0.1015), ('d', 0.056), ('e', 0.0466), ('f', 0.0432), ...]
E    +  where None = <function solution_exercise6 at 0x7fcac73767a0>([('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ...])
E    +  and   [('a', 0.072), ('b', 0.0536), ('c', 0.1015), ('d', 0.056), ('e', 0.0466), ('f', 0.0432), ...] = reference_solution_exercise6([('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ...])
=========================== short test summary info ============================
FAILED tutorial/tests/test_f

### Exercise 7: Finding palindromes 🌶️🌶️🌶️
Consider again the `words` list of strings. Write a function  that returns the list of all *palindromes*. A *palindrome* is a word (any string longer than 1) that reads the same left-to-right and right-to-left;
for example:
- rotator
- wow
- noon

As usual, the words are available as the input `words` to `solution_exercise7`

In [8]:
%%ipytest functional_programming
def solution_exercise7(words: "list[str]") -> "list[str]":
    pass


F                                                                        [100%]
=================================== FAILURES ===================================
______________________ test_exercise7[solution_exercise7] ______________________
tutorial/tests/test_functional_programming.py:92: in test_exercise7
    assert function_to_test(data) == reference_function_exercise7(data)
E   AssertionError: assert None == ['a', 'aa', 'aaa', 'ada', 'aka', 'ala', ...]
E    +  where None = <function solution_exercise7 at 0x7f96bff81080>(['a', 'aa', 'aaa', 'aaron', 'ab', 'abandoned', ...])
E    +  and   ['a', 'aa', 'aaa', 'ada', 'aka', 'ala', ...] = reference_function_exercise7(['a', 'aa', 'aaa', 'aaron', 'ab', 'abandoned', ...])
=========================== short test summary info ============================
FAILED tutorial/tests/test_functional_programming.py::test_exercise7[solution_exercise7] - AssertionError: assert None == ['a', 'aa', 'aaa', 'ada', 'aka', 'ala', ...]
1 failed in 0.01s

